In [1]:
#Install needed libraries
!pip install librosa
!pip install numpy
!pip install scipy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install torch
!pip install yt-dlp
!pip install google-api-python-client
!pip install isodate
!pip install tempfile
!pip install fuzzywuzzy
!pip install python-Levenshtein

ERROR: Could not find a version that satisfies the requirement tempfile (from versions: none)
ERROR: No matching distribution found for tempfile


In [2]:
#Import Libraries
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import yt_dlp
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.signal import find_peaks
from scipy.spatial.distance import cosine
from googleapiclient.discovery import build
import time
import random 
from googleapiclient.errors import HttpError
import isodate 
from tempfile import NamedTemporaryFile #for Streamlit app
import streamlit as st #streamlit app
import joblib #streamlit/parallel jobs if needed
import os #for saving my key in an env var
from dotenv import load_dotenv #to load that env var
import re #regex for fuzzy lookup in score titles
from fuzzywuzzy import fuzz #fuzzy match for score titles

In [3]:
#Get credentials for youtube data API
load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")  
YOUTUBE_API_SERVICE_NAME = "youtube" #This should be a set feature (stay at youtube and just make sure I don't need to create a service account)
YOUTUBE_API_VERSION = "v3" #Check under google's api service feature and make sure I check the box to enable the api latest version

In [4]:
#create a list of composers I want to parse through on youtube
composers = [
    "Hans Zimmer", "John Williams", "James Newton Howard", 
    "Howard Shore", "Danny Elfman", "Ennio Morricone", 
    "Alexandre Desplat", "Thomas Newman", "Michael Giacchino",
    "John Powell", "Harry Gregson-Williams", "Rupert Gregson-Williams",
    "James Horner", "Lorne Balfe", "Kevin Kiner", "Henry Jackman",
    "Dominic Lewis", "Patrick Doyle", "Nicholas Hooper",
    "Joe Kraemer", "David Arnold", "John Barry", "Eric Serra",
    "John Debney", "Brian Tyler", "Alan Silvestri",
    "Mark Mothersbaugh", "Christophe Beck", "Joel P West",
    "Ludwig Goransson", "Randy Newman", "Daniel Pemberton",
    "Jerry Goldsmith", "Randy Edelman", "John Ottman",
    "Mark Isham", "Alan Menken", "Klaus Badelt",
    "Michael Kamen", "Marc Shaiman", "Trevor Rabin", 
    "Gavin Greenaway", "Justin Hurwitz",
    "Trent Reznor and Atticus Ross", "Trevor Jones",
    "Elmer Bernstein", "Carlos Rafael Rivera", "Don Davis",
    "Joe Hisaishi", "Tan Dun", "Dave Grusin",
    "Dario Marianelli", "Steve Jablonsky", "Bill Conti",
    "Ramin Djawadi", "Anthony Gonzalez", "Mychael Danna",
    "Johnny Klimek", "Tom Tykwer", "Michael Paraskevas",
    "Harold Faltermeyer", "Natalie Holt", "James Shearman",
    "Robin Carolan", "Tyler Bates", "Bear McCreary",
    "Nicholas Britell", "Carter Burwell", "Martin Phipps",
    "Naoki Sato", "Takeshi Furukawa", "John Paesano",
    "Benjamin Wallfisch", 
]

In [5]:
personal_projects_path = r"G:\My Drive\Personal Projects" #update my path to my personal project folder
if personal_projects_path not in sys.path:
    sys.path.append(personal_projects_path)

os.chdir(personal_projects_path) 
print(f"✅ Now working in: {os.getcwd()}")

✅ Now working in: G:\My Drive\Personal Projects


In [6]:
#Info on requests:
#The search request counts as one API call per page of results.
#The video details request is made for each video returned in the search results.
#Since each page fetches up to 25 videos, that means each full page of results adds ~26 requests (1 search request + 25 video details requests).


#Request Tracker
##MAX_DAILY_REQUESTS = 10000
##requests_made = 0

#Create a file to store video urls
##URL_FILE = r"video_urls.txt"
##full_path = os.path.abspath(URL_FILE)

#Set a max number of files for each artist (we will update this after each full iter so once everyone has 5 we will move to 10 etc
##MAX_VIDEOS = 5

#Read the existing files for each artist
##video_counts = {composer: 0 for composer in composers}
##existing_videos = {}
#Count the number of unique files for each artist
##if os.path.exists(URL_FILE):
##    with open(URL_FILE, "r", encoding="utf-8") as f:
##        for line in f:
##            composer, url = line.strip().split(" | ")
##            existing_videos.setdefault(composer, []).append(url)
##            video_counts[composer] += 1

#Create a function to fetch videos
##def fetch_composer_ost_links(composer, max_videos=5):
##    global requests_made

##    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)
##    query = f"{composer} ost"
##    links = []
##    next_page_token = None
#Define keywords we need to not have pulled in our videos...filter out all of these (Remember to add Official Video, Scoring Session and Extended)
##    exclude_keywords = [
##        "live", "interview", "trailer", "behind the scenes", "making of",
##        "creating", "composing", "inspiration", "mix", "remix", "medley",
##        "suite", "score analysis", "cover of", "analysis", "analyzing",
##        "producing", "orchestrating", "orchestration", "recording", "concert",
##        "Top 5", "top 5", "Top 10", "top 10", "Best of", "best of", "Boston Pops", "boston pops",
##        "Greatest Hits", "greatest hits", "Best Pieces", "best pieces", "Best Works", "best works", 
##        "Official Video", "official video", "Scoring Session", "scoring session", "Extended", "extended",
##        "sad part", "Sad Part", "Happy Part", "happy part", "the best scores", "The Best Scores",
##        "The Best Soundtracks", "the best soundtracks", "EDITED", "edited", "Edited", "Soundtrack Review", "soundtrack review",
##        "Score Review", "score review", "Epic Version", "epic version", "Score Highlights", "score highlights",
##        "Soundtrack Highlights", "soundtrack highlights", "arr.", "Arr.", "Cover", "cover"
##    ]
#while we are unde the api daily limits
##    while len(links) < max_videos and requests_made + 1 <= MAX_DAILY_REQUESTS:
##        if requests_made >= MAX_DAILY_REQUESTS:
##            print("🚨 API limit reached, stopping collection.")
##            break

        # Search request (counts as 1 request)
##        search_response = youtube.search().list(
##            q=query,
##            part="id,snippet",
##            type="video",
##            maxResults=min(25, max_videos - len(links)),
##            pageToken=next_page_token
##        ).execute()
##        requests_made += 1  # Track search request

##        time.sleep(random.uniform(1, 3))  # Sleep timer between requests

##        video_ids = []
##        for item in search_response.get("items", []):
##            title = item["snippet"]["title"].lower()
##            video_id = item["id"]["videoId"]
##            video_url = f"https://www.youtube.com/watch?v={video_id}" #Tell me the name of the url

##            if video_url in existing_videos.get(composer, []) or video_url in links:
##                continue  #No duplicate tracks
            
##            if not any(keyword in title for keyword in exclude_keywords):
##                if any(term in title for term in ["ost", "original motion picture", "soundtrack", "original score", "score"]):
##                    video_ids.append(video_id)  #Collect video IDs for batch request

##        if not video_ids:
##            next_page_token = search_response.get("nextPageToken")
##            if not next_page_token:
##                break  #Stop if no more pages
##            continue

        #Video details request (counts as len(video_ids) requests)
##        try:
##            video_response = youtube.videos().list(
##                part="contentDetails",
##                id=",".join(video_ids)  #Batch request for efficiency (looking into this)
##            ).execute()
##            requests_made += len(video_ids)  
##        except Exception as e:
##            print(f"⚠️ Error fetching video details: {e}") #if there's an error with video details skip the song
##            continue

##        for item in video_response.get("items", []):
##            video_id = item["id"]
##            video_url = f"https://www.youtube.com/watch?v={video_id}" #the actual data we're pulling
##            content_details = item.get("contentDetails", {})

##            if "duration" not in content_details:
##                print(f"⚠️ Skipping {video_url} (Missing duration info)") #if there's no video duration info, skip the song
##                continue

##            duration = content_details["duration"]
##            duration_seconds = isodate.parse_duration(duration).total_seconds()

##            if 60 <= duration_seconds <= 12 * 60:  #12 minute maximum video length
##                links.append(video_url)
##                print(f"✅ Added: {video_url} ({duration_seconds // 60} min)")

##        next_page_token = search_response.get("nextPageToken")
##        if not next_page_token:
##            break  #if you run out of pages stop

##        time.sleep(random.uniform(5, 15))  #sleep delay

##    return links


#find videos for composers that haven't reached max
##for composer in composers:
##    if video_counts[composer] >= MAX_VIDEOS:
##        print(f"⏭️ Skipping {composer}, already has {MAX_VIDEOS} videos.")
##        continue  #kkip composers with max 

##    try:
##        print(f"🎼 Fetching {MAX_VIDEOS - video_counts[composer]} tracks for {composer}...") #tell me when you move onto a new artist
##        composer_links = fetch_composer_ost_links(composer, max_videos=MAX_VIDEOS - video_counts[composer])

##        new_urls = [f"{composer} | {url}" for url in composer_links]
##        video_counts[composer] += len(new_urls)

        # Append new URLs to the file
##        with open(URL_FILE, "a", encoding="utf-8") as f:
##            for url in new_urls:
##                f.write(url + "\n")

##        print(f"✅ Added {len(new_urls)} tracks for {composer}") #tell me you're adding tracks for each artist

##    except Exception as e:
##        print(f"⚠️ Error with {composer}: {e}") #if there's an error with certain artists let's pull them
#Tell me where we are saving data at the end
##print(f"📂 URLs saved to: {full_path}")

# Request Tracker
MAX_DAILY_REQUESTS = 10000
requests_made = 0

# Create a file to store video URLs
URL_FILE = r"video_urls.txt"
full_path = os.path.abspath(URL_FILE)

# Set max videos per composer
MAX_VIDEOS = 5

# Read existing videos from the text file
video_counts = {composer: 0 for composer in composers}
existing_videos = {}
existing_titles = {}

if os.path.exists(URL_FILE):
    try:
        with open(URL_FILE, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    composer, url = line.strip().split(" | ")
                    existing_videos.setdefault(composer, []).append(url)
                    video_counts[composer] += 1

                    title = url.split("?v=")[-1]  # Extract video ID
                    existing_titles.setdefault(composer, []).append(title)
                except ValueError:
                    print(f"⚠️ Skipping invalid line: {line.strip()}")
    except Exception as e:
        print(f"⚠️ Error reading {URL_FILE}: {e}")

#Crated a function to normalize all film score track titles
def normalize_title(title):
    title = title.lower()  #make titles all lowercase
    title = re.sub(r"[^a-z0-9\s]", "", title)  #get rid of any special characters
    title = re.sub(r"\s+", " ", title).strip()  #strip white space
    return title

def fetch_composer_ost_links(composer, max_videos=5):
    global requests_made

    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY) #api request itself
    query = f"{composer} ost" #simply look for one of the composers in our list and OST next to their name as an initial search
    links = [] #create an empty list
    next_page_token = None

    exclude_keywords = [ #created a list of words we want to avoid (these pertain to tracks that aren't official (check on osts and soundtracks to make sure it doesn't break)
        "live", "interview", "trailer", "behind the scenes", "making of",
        "creating", "composing", "inspiration", "mix", "remix", "medley",
        "suite", "score analysis", "cover of", "analysis", "analyzing",
        "producing", "orchestrating", "orchestration", "recording", "concert",
        "Top 5", "top 5", "Top 10", "top 10", "Best of", "best of", "Boston Pops", "boston pops",
        "Greatest Hits", "greatest hits", "Best Pieces", "best pieces", "Best Works", "best works", 
        "Official Video", "official video", "Scoring Session", "scoring session", "Extended", "extended",
        "sad part", "Sad Part", "Happy Part", "happy part", "the best scores", "The Best Scores",
        "The Best Soundtracks", "the best soundtracks", "EDITED", "edited", "Edited", "Soundtrack Review", "soundtrack review",
        "Score Review", "score review", "Epic Version", "epic version", "Score Highlights", "score highlights",
        "Soundtrack Highlights", "soundtrack highlights", "arr.", "Arr.", "Cover", "cover", "edit", "Edit", "OSTS", "osts", "Soundtracks"
        "soundtracks", "osts"
    ]

    while len(links) < max_videos and requests_made + 1 <= MAX_DAILY_REQUESTS: #while we are under the daily API requests limit
        if requests_made >= MAX_DAILY_REQUESTS:
            print("🚨 API limit reached, stopping collection.") #if not give me a warning when we hit it (or you'll see an error 403)
            break #stop searching

        search_response = youtube.search().list( #create a youtube query 
            q=query,
            part="id,snippet",
            type="video",
            maxResults=min(25, max_videos - len(links)), #max of 25 videos per page(we can look into this to see if expanding is the best idea)
            pageToken=next_page_token #if we don't pull anything new on that page move to the enxt page
        ).execute()
        requests_made += 1 #update the api requests tracker (1 per page pulled and 1 per video pulled) right now 26 per page

        time.sleep(random.uniform(1, 3)) #random sleep delay to not get flagged

        video_ids = [] #empty list to store video ids
        video_titles = [] #empty list to store titles
        for item in search_response.get("items", []):
            title = item["snippet"]["title"]
            video_id = item["id"]["videoId"]
            video_url = f"https://www.youtube.com/watch?v={video_id}" #print out the link for the video (so we can manually check)

            normalized_title = normalize_title(title) #apply the normalize function to each video title so we can then compare to videos in our txt file

            # Skip duplicate URLs
            if video_url in existing_videos.get(composer, []) or video_url in links: #if we already have that url in our .txt file, don't grab it
                continue
            
            # Skip duplicate/similar titles
            if composer in existing_titles:
                if any(fuzz.ratio(normalized_title, existing) > 95 for existing in existing_titles[composer]): #apply fuzzy ratio (currently at 95% to known video titles)
                    print(f"⚠️ Skipping duplicate title: {title}") #tell me if a track is too similar and we should skip over it
                    continue

            if not any(keyword in title.lower() for keyword in exclude_keywords): #if there's no exclude words in the track title
                if any(term in title.lower() for term in ["ost", "original motion picture", "soundtrack", "original score", "score"]): #our key buzz words
                    video_ids.append(video_id) #grab the id and add it 
                    video_titles.append(normalized_title) #grab the normalized title and add it

        if not video_ids: #if there's no videos in the video id object
            next_page_token = search_response.get("nextPageToken") #move to the next page of videos on youtube
            if not next_page_token: #if there's no next page and we run out of relavant videos...stop the script
                break
            continue

        try:
            video_response = youtube.videos().list(
                part="contentDetails",
                id=",".join(video_ids)
            ).execute()
            requests_made += len(video_ids)
        except Exception as e:
            print(f"⚠️ Error fetching video details: {e}") #tell me if there's an issue grabbing video content data
            continue

        for i, item in enumerate(video_response.get("items", [])):
            video_id = item["id"]
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            content_details = item.get("contentDetails", {})

            if "duration" not in content_details:
                print(f"⚠️ Skipping {video_url} (Missing duration info)") #tell me if there's an issue grabbing video time details
                continue

            duration = content_details["duration"]
            duration_seconds = isodate.parse_duration(duration).total_seconds()

            if 60 <= duration_seconds <= 12 * 60:
                links.append(video_url)
                existing_titles.setdefault(composer, []).append(video_titles[i])  #Store normalized title
                print(f"✅ Added: {video_url} ({duration_seconds // 60} min)") #tell me how long the track is

        next_page_token = search_response.get("nextPageToken")
        if not next_page_token:
            break

        time.sleep(random.uniform(5, 15)) #another time delay in between videos

    return links

#Find videos for composers that haven't reached max
for composer in composers:
    if video_counts.get(composer, 0) >= MAX_VIDEOS:
        print(f"⏭️ Skipping {composer}, already has {MAX_VIDEOS} videos.")
        continue  #Skip composers with self imposed max

    try:
        remaining_videos = MAX_VIDEOS - video_counts[composer]
        print(f"🎼 Fetching {remaining_videos} tracks for {composer}...")  #Tell if videos are left to pull for incomplete composers
        composer_links = fetch_composer_ost_links(composer, max_videos=remaining_videos)

        new_urls = [f"{composer} | {url}" for url in composer_links]
        video_counts[composer] += len(new_urls)

        #Add new URLs to the file
        try:
            with open(URL_FILE, "a", encoding="utf-8") as f:
                for url in new_urls:
                    f.write(url + "\n")
        except Exception as e:
            print(f"⚠️ Error writing to file: {e}")

        print(f"✅ Added {len(new_urls)} tracks for {composer}")  #Tell me how many new videos you added for each atrist (if new)

    except Exception as e:
        print(f"⚠️ Error with {composer}: {e}") #tell me if there's an error for each artist (So I know if I have to pull them)

# Log where data is saved
print(f"📂 URLs saved to: {full_path}") #make sure the data is saved back to our Google drive video urls.txt file

⏭️ Skipping Hans Zimmer, already has 5 videos.
⏭️ Skipping John Williams, already has 5 videos.
⏭️ Skipping James Newton Howard, already has 5 videos.
⏭️ Skipping Howard Shore, already has 5 videos.
⏭️ Skipping Danny Elfman, already has 5 videos.
⏭️ Skipping Ennio Morricone, already has 5 videos.
⏭️ Skipping Alexandre Desplat, already has 5 videos.
⏭️ Skipping Thomas Newman, already has 5 videos.
⏭️ Skipping Michael Giacchino, already has 5 videos.
⏭️ Skipping John Powell, already has 5 videos.
⏭️ Skipping Harry Gregson-Williams, already has 5 videos.
⏭️ Skipping Rupert Gregson-Williams, already has 5 videos.
⏭️ Skipping James Horner, already has 5 videos.
⏭️ Skipping Lorne Balfe, already has 5 videos.
⏭️ Skipping Kevin Kiner, already has 5 videos.
⏭️ Skipping Henry Jackman, already has 5 videos.
⏭️ Skipping Dominic Lewis, already has 5 videos.
⏭️ Skipping Patrick Doyle, already has 5 videos.
⏭️ Skipping Nicholas Hooper, already has 5 videos.
⏭️ Skipping Joe Kraemer, already has 5 vide